In [2]:
import json, os, re, shutil, sys, time
import collections, itertools
import unittest
from IPython.display import display, HTML

# NLTK for NLP utils and corpora
import nltk

# NumPy and TensorFlow
import numpy as np
import tensorflow as tf
assert(tf.__version__.startswith("1."))

# utils.pretty_print_matrix uses Pandas. Configure float format here.
import pandas as pd
pd.set_option('float_format', lambda f: "{0:.04f}".format(f))

# RNNLM Model
import rnnlm
reload(rnnlm)

<module 'rnnlm' from 'rnnlm.pyc'>

Vocabulary class holds the vocabulary and the mapping between words and ids for the words.

In [3]:
import collections
'''Vocabulary class, nearly identical to that used in a4'''
class Vocabulary(object):

  UNK_TOKEN = "<unk>"

  def __init__(self, tokens, size=None):
    self.unigram_counts = collections.Counter(tokens)
    # leave space for "<unk>"
    top_counts = self.unigram_counts.most_common(None if size is None else (size - 1))
    vocab = ([self.UNK_TOKEN] +
             [w for w,c in top_counts])

    # Assign an id to each word, by frequency
    self.id_to_word = dict(enumerate(vocab))
    self.word_to_id = {v:k for k,v in self.id_to_word.iteritems()}
    self.size = len(self.id_to_word)
    if size is not None:
        assert(self.size <= size)

    # For convenience
    self.wordset = set(self.word_to_id.iterkeys())

    # Store special IDs
    self.UNK_ID = self.word_to_id[self.UNK_TOKEN]

  def words_to_ids(self, words):
    return [self.word_to_id.get(w, self.UNK_ID) for w in words]

  def ids_to_words(self, ids):
    return [self.id_to_word[i] for i in ids]

  def ordered_words(self):
    """Return a list of words, ordered by id."""
    return self.ids_to_words(range(self.size))


These functions are used to massage the cleaned data into indivdual words. Punctuation at the end of a word is split into its own distince word. Also accomplishes some minor data cleaning, removing nonsense characters.

In [4]:
from os import listdir
import re

def canonicalize_digits(word):
    if any([c.isalpha() for c in word]): return word
    word = re.sub("\d", "DG", word)
    if word.startswith("DG"):
        word = word.replace(",", "") # remove thousands separator
    return word

def canonicalize_word(word):
    word = word.lower()
    return canonicalize_digits(word) # try to canonicalize numbers

def replace_all(text, dic):
    for i, j in dic.iteritems():
        text = text.replace(i, j)
    return text

def canonicalize_words(words):
    current = []
    rep_dict = {'\n':' '
                ,'\xc2':' '
                ,'\xa0':' '
                ,'\xc2':' '
                ,'\xc3':' '
                ,'\xa9':' '
                ,'\xef':' '
                ,'\xbb':' '
                ,'\xbf':' '
                ,'\xa6':' '
                ,'\xb9':' '
                ,'\xa3':' '
                ,'\xbd':' '
                ,'\xb4':' '
                ,'\xcb':' '
                ,'\x9a':' '
                ,'\x86':' '
                ,'\xcf':' '
                ,'\x84':' '
                ,'\xce':' '
                ,'\x87':' '
                ,'\xe2':' '
                ,'\x80':' '
                ,'\x94':' '
               }
    for word in replace_all(words, rep_dict).split(' '):   
        if word:
            if word[-1] in ('.', ',', '?', ';', '!'):
                punk = word[-1]
                current.append(punk)
                word = word[0:-1]

            word = canonicalize_word(word)
            current.append(word)
    return current

In [5]:
def pretty_timedelta(fmt="%d:%02d:%02d", since=None, until=None):
    """Pretty-print a timedelta, using the given format string."""
    since = since or time.time()
    until = until or time.time()
    delta_s = until - since
    hours, remainder = divmod(delta_s, 3600)
    minutes, seconds = divmod(remainder, 60)
    return fmt % (hours, minutes, seconds)

load_train_data reads in everything in train_data directory from which it generates the vocab (of type Vocabulary defined above), the author_to_id map (which maps author names to ids) and two arrays, one contains the text of each file (as a list of word ids) under "train_data". The other holds the author id for each of the texts.

In [6]:
def load_train_data():
    train_data_dir = 'train_data'
    y = []
    X = []
    all_tokens = []
    author_to_id = {}
    for author_id, author in enumerate(listdir(train_data_dir)):
        author_to_id[author] = author_id
        author_path = "%s/%s" % (train_data_dir, author)
        print author, author_id

        for file_name in listdir(author_path):
            full_path = "%s/%s" % (author_path, file_name)
            y.append(author_id)            
            with open(full_path, "r") as f:
                current = canonicalize_words(f.read())
                all_tokens += current
                X.append(np.array(current))
                
    vocab = Vocabulary(all_tokens)

    # replace words with ids
    for i, x in enumerate(X):
        # X[i] = np.array(x) # This line can be used to make sure your words are useful 
        X[i] = np.array(vocab.words_to_ids(x))

    return vocab, np.array(X), np.array(y), author_to_id


def id_to_author(author_to_id, id):
    for author, author_id in author_to_id.iteritems():
        if id == author_id:
            return author

Here we load the training data, note resulting number of classes (authors) and display some useful information.

In [7]:
vocab, X_train, y_train, author_to_id = load_train_data()
num_classes = len(np.unique(y_train))
print "vocab.size", vocab.size
print author_to_id

thomas_paine 0
thomas_jefferson 1
john_adams 2
james_madison 3
alexander_hamilton 4
james_monroe 5
john_jay 6
george_washington 7
benjamin_franklin 8
vocab.size 48471
{'thomas_jefferson': 1, 'john_adams': 2, 'alexander_hamilton': 4, 'benjamin_franklin': 8, 'george_washington': 7, 'thomas_paine': 0, 'james_madison': 3, 'james_monroe': 5, 'john_jay': 6}


Load the eval data in the same format as the training data. But each federalist paper here ends up in its own dictionary entry so that they can be scored/classified/attributed separately. Each is assumed to be written by James Madison.

In [8]:
def load_eval_data(vocab, eval_data_dir):
    eval_X = {}
    eval_y = {}
    for file_name in listdir(eval_data_dir):
        full_path = "%s/%s" % (eval_data_dir, file_name)
        with open(full_path, "r") as f:
            current = vocab.words_to_ids(canonicalize_words(f.read()))

        expanded_X = np.array(current)
        id = file_name.split("_")[2].split(".")[0]
        eval_X[id] = np.array([expanded_X])
        # working with the assumption that James Madison wrote all the disputed papers
        if eval_data_dir == "unknown_data":
            eval_y[id] = np.array([author_to_id['james_madison']])
        else:
            if id == '5':
                eval_y[id] = np.array([author_to_id['john_jay']])
            elif id == '39':
                eval_y[id] = np.array([author_to_id['james_madison']])
            else:
                eval_y[id] = np.array([author_to_id['alexander_hamilton']])
                
    return eval_X, eval_y

eval_X, eval_y = load_eval_data(vocab, "unknown_data")
print eval_y['18']
print eval_X['18']

test_X, test_y = load_eval_data(vocab, "test_data")
print "Who wrote Federalist paper 5 (John Jay should be answer): %s" % id_to_author(author_to_id, test_y['5'])

[3]
[[ 180    1 4299 ...,  398    5  500]]
Who wrote Federalist paper 5 (John Jay should be answer): john_jay


Cut up the publications based on batch_size, and max_time. To reduce how much code had to be changed from a4 this expands the author id for each document to be an author id for each word in the document. So if you had publication[1] = [1 2 3] and authors[1] = 1, this would output (assuming a batch of 1 and max time of 3) w = [1 2 3] and y = [1 1 1]. In the end we'll ignore all the loss for everything except the last word, but all the matrix functions and multiplications could work as is if I kept expanded the author to be associated with each word (since that is what the sequence math was doing, each word had a corresponding target word). Note: this also randomly shuffles the batches so that an given author's data is mixed through out the training process.

In [9]:
def slice_up_words(words, window_size=10, step_size=1):
    clip_len = ((len(words)-1) / window_size) * window_size
    words = words[:clip_len]
    slices = []
    num_words = len(words)
    for index in range(0, num_words, step_size):
        slices.append(words[index:index+window_size])
    return slices
        
slice_up_words(["hello", "I", "am", "mr", ".", "anderson", "what", "is", "your", "name", "?"], 3, 3)

[['hello', 'I', 'am'], ['mr', '.', 'anderson'], ['what', 'is', 'your']]

In [10]:
import random
# maybe worth seeding here. or in the batch_generator function

def shape_data_for_batching(publications, authors, max_time):
    """Convert ids to data-matrix form."""
    all_w = None
    all_y = []
    for i, ids in enumerate(publications):
        ids = np.array(slice_up_words(ids, max_time, max_time))
        y = np.full_like(ids, authors[i])
        
        if all_w is not None:
            all_w = np.append(all_w, ids, 0)
            all_y = np.append(all_y, y, 0)
            
        else:
            all_w = ids
            all_y = y

    # Yield batches in random order     
    index = range(0, len(all_y)-1)
    random.shuffle(index)   

    all_w = [all_w[i] for i in index]
    all_y = [all_y[i] for i in index]

    return all_w, all_y

def batch_generator(X_shaped, y_shaped, batch_size):
    clip_len = ((len(X_shaped)-1) / batch_size) * batch_size
    X_shaped = X_shaped[:clip_len]
    y_shaped = y_shaped[:clip_len]  
    for j in xrange(0, len(X_shaped), batch_size):
        this_x = X_shaped[j:j+batch_size]
        this_y = y_shaped[j:j+batch_size]
        yield np.array(this_x), np.array(this_y)


In [11]:
# Sanity Check
X_shaped, y_shaped = shape_data_for_batching(X_train, y_train, 5)

In [12]:
for i, (w, y) in enumerate(batch_generator(X_shaped, y_shaped, 10)):
    print y
    print w 
    break

[[1 1 1 1 1]
 [1 1 1 1 1]
 [3 3 3 3 3]
 [6 6 6 6 6]
 [1 1 1 1 1]
 [3 3 3 3 3]
 [1 1 1 1 1]
 [4 4 4 4 4]
 [1 1 1 1 1]
 [4 4 4 4 4]]
[[   96   842    26     1    97]
 [    1 13710  1809     2 11606]
 [    1   378     3 20161 10479]
 [   28     5  3535    65     6]
 [  127     8  3050  3717     3]
 [ 1781     5  1378   113   139]
 [  132    17     1   552     3]
 [ 2134     3  4582   112   733]
 [  915    16     8  7779     6]
 [    7  1848    19    70  8850]]


Runs the epoch. Mostly the same as a4, but there is no test phase (instead there is no a separate prediction phase) that didn't makes as much sense to me to have in this function, so it has its own function

In [13]:
def run_epoch(lm, session, batch_iterator,
              train=False, verbose=False,
              tick_s=10, learning_rate=0.1):
    start_time = time.time()
    tick_time = start_time  # for showing status
    total_cost = 0.0  # total cost, summed over all words
    total_batches = 0
    total_words = 0

    if train:
        train_op = lm.train_step_
        use_dropout = True
    else:
        train_op = tf.no_op()
        use_dropout = False  # no dropout at test time

    for i, (w, y) in enumerate(batch_iterator):
        cost = 0.0
        # At first batch in epoch, get a clean intitial state.
        if i == 0:
            h = session.run(lm.initial_h_, {lm.input_w_: w})
            
        feed_dict = {lm.input_w_: w,
                     lm.target_y_: y,
                     lm.initial_h_: h,
                     lm.learning_rate_: learning_rate,
                     lm.use_dropout_: use_dropout}
        
        _, h, cost = session.run([train_op, lm.final_h_, lm.loss_], feed_dict)  

        total_cost += cost
        total_batches = i + 1
        total_words += w.size  # w.size = batch_size * max_time

        ##
        # Print average loss-so-far for epoch
        # If using train_loss_, this may be an underestimate.
        if verbose and (time.time() - tick_time >= tick_s):
            avg_cost = total_cost / total_batches
            avg_wps = total_words / (time.time() - start_time)
            print "[batch %d]: seen %d words at %d wps, loss = %.3f" % (
                i, total_words, avg_wps, avg_cost)
            tick_time = time.time()  # reset time ticker

    return total_cost / total_batches

In [14]:
def score_dataset(lm, session, ids, authors, max_time, name="Data"):
    # For scoring, we can use larger batches to speed things up. Same as a4
    bi = batch_generator(ids, authors, batch_size=100, max_time=max_time)
    cost = run_epoch(lm, session, bi, 
                     learning_rate=1.0, train=False, 
                     verbose=False, tick_s=3600)
    print "%s: avg. loss: %.03f  (perplexity: %.02f)" % (name, cost, np.exp(cost))
    return cost

Functions used to predict a batch iterators data. Used post training to test the unknown federalist papers.

In [29]:
from collections import defaultdict

def get_prediction_results(predictions, author_to_id, print_results=True):
    '''
    Takes the predictions for a set of text batches and calculates the percentage
    of patches predicted for each author.
    '''
    counts = defaultdict(float)
    for p in predictions:
        counts[p] += 1

    # getting prediction percentages to return for later viewing
    predictions_num = len(predictions)
    results_dict = {author_id:float(count)/predictions_num for author_id, count in counts.iteritems()}
    
    if print_results:
        # prints results if indicated
        print "Prediction Summary:"
        for author_id, prediction in results_dict.iteritems():
            print "%s: %.2f" % (id_to_author(author_to_id, author_id), prediction)
        print ""

    return results_dict
    
def predict_paper(lm, session, batch_iterator, authors, paper_name, print_results=True):
    '''
    Splits given paper into batches and predicts the author for each batch.
    Passes these predictions to get_prediction_results() to tally the batches for each author
    and arrive at a final prediction percentage.
    '''
    total_predictions = np.array([])
    if print_results:
        print "Predicting for %s" % paper_name
    for i, (w, y) in enumerate(batch_iterator):        
        if i == 0 and print_results:
            print "Truth:", id_to_author(authors, y[0][0])
            
        feed_dict = {lm.input_w_: w,
                     lm.target_y_: y}
        
        cost, truths, logits, predictions = session.run([lm.loss_, lm.target_y_last_, lm.logits_last_, lm.predictions_], feed_dict)  
        total_predictions = np.append(total_predictions, predictions.reshape(-1))

    # gets final predictions, by author, for the given paper
    results_dict = get_prediction_results(total_predictions, authors, print_results=print_results)
    return results_dict

def test_papers(lm, session, papers, labels, authors, batch_size, print_results=True):
    '''
    Predicts authorship for given papers.
    Returns a dictionary of dictionaries containing the predictions by author for each paper.
    '''
    full_results = dict()
    for key in papers:
        prediction_bi = batch_generator(papers[key], labels[key], batch_size)
        paper_results = predict_paper(lm, session, prediction_bi, authors, "Federalist Paper %s" % key, print_results=print_results)
        full_results[key] = paper_results
    
    return full_results

def create_predictions_dataframe(epoch_predictions):
    '''
    Input: List of predictions for each epoch, where an epochs predictions is a dictionary of dictionaries containing
    the predictions for each paper by author.
    Output: Pandas dataframe of results for each author by paper, with a flag indicating the epoch.
    '''
    df_list = []
    for epoch in range(len(epoch_predictions)):
        epoch_num = epoch + 1 # we indexed our epochs from 1, but this list is 0-indexed of course
        df = pd.DataFrame(epoch_predictions[epoch])
        df['epoch'] = epoch_num
        df_list.append(df)

    return pd.concat(df_list)
    
get_prediction_results([1, 2, 1, 2], author_to_id)

Prediction Summary:
thomas_jefferson: 0.50
john_adams: 0.50



{1: 0.5, 2: 0.5}

In [30]:
# Training parameters
max_time = 15
batch_size = 40
learning_rate = 0.1
num_epochs =3

save_predictions = True # flag to determine if we save the predictions on unkown papers in each epoch

# Model parameters
model_params = dict(V=vocab.size, 
                    H=100, 
                    num_classes=num_classes,
                    num_layers=1)

TF_SAVEDIR = "tf_saved"
checkpoint_filename = os.path.join(TF_SAVEDIR, "rnnlm")
trained_filename = os.path.join(TF_SAVEDIR, "rnnlm_trained")


X_train_shaped, y_train_shaped = shape_data_for_batching(X_train, y_train, max_time)

X_test_shaped = {}
y_test_shaped = {}
for key in test_X:
    X_test_shaped[key], y_test_shaped[key] = shape_data_for_batching(test_X[key], test_y[key], max_time)

X_eval_shaped = {}
y_eval_shaped = {}
for key in eval_X:
    X_eval_shaped[key], y_eval_shaped[key] = shape_data_for_batching(eval_X[key], eval_y[key], max_time)


Nearly the same as a4, but instead of scoring the data we look at prediction results after each epoch.

In [31]:
reload(rnnlm)
# Will print status every this many seconds
print_interval = 5

# Clear old log directory
shutil.rmtree("tf_summaries", ignore_errors=True)

lm = rnnlm.RNNLM(**model_params)
lm.BuildCoreGraph()
lm.BuildTrainGraph()
lm.BuildClassifierGraph()

# Explicitly add global initializer and variable saver to LM graph
with lm.graph.as_default():
    initializer = tf.global_variables_initializer()
    saver = tf.train.Saver()
    
# Clear old log directory
shutil.rmtree(TF_SAVEDIR, ignore_errors=True)
if not os.path.isdir(TF_SAVEDIR):
    os.makedirs(TF_SAVEDIR)

with tf.Session(graph=lm.graph) as session:
    # Seed RNG for repeatability
    tf.set_random_seed(42)

    session.run(initializer)

    # List for saving the unkown paper predictions from each epoch
    if save_predictions:
        epoch_predictions = []
        
    for epoch in xrange(1,num_epochs+1):
        t0_epoch = time.time()
        bi = batch_generator(X_train_shaped, y_train_shaped, batch_size)
        print "[epoch %d] Starting epoch %d" % (epoch, epoch)
        # Run a training epoch.

        run_epoch(lm, session, bi, train=True, verbose=True, tick_s=100, learning_rate=learning_rate)
    
        print "[epoch %d] Completed in %s" % (epoch, pretty_timedelta(since=t0_epoch))
    
        # Save a checkpoint
        saver.save(session, checkpoint_filename, global_step=epoch)
    
        ##
        # score_dataset will run a forward pass over the entire dataset
        # and report perplexity scores. This can be slow (around 1/2 to 
        # 1/4 as long as a full epoch), so you may want to comment it out
        # to speed up training on a slow machine. Be sure to run it at the 
        # end to evaluate your score.
        print ("[epoch %d]" % epoch),
        #score_dataset(lm, session, eval_X['18'], eval_y['18'], max_time, name="Federalist Paper 18")
        #score_dataset(lm, session, test_ids, name="Test set")

        # test three of the federalist papers whose author is known after each epoch
        test_predictions = test_papers(lm, session, X_test_shaped, y_test_shaped, author_to_id, batch_size, print_results=True)
        
        if save_predictions:
            # testing against all unkown federalist papers
            unk_predictions = test_papers(lm, session, X_eval_shaped, y_eval_shaped, author_to_id, batch_size, print_results=False)
            epoch_predictions.append(unk_predictions)

    # Testing final model against all unkown federalist papers
    final_test = test_papers(lm, session, X_eval_shaped, y_eval_shaped, author_to_id, batch_size, print_results=True)    
    # Save final model
    saver.save(session, trained_filename)
    


[epoch 1] Starting epoch 1
[batch 4720]: seen 2832600 words at 28323 wps, loss = 1.395
[epoch 1] Completed in 0:02:16
[epoch 1] Predicting for Federalist Paper 39
Truth: james_madison
Prediction Summary:
thomas_jefferson: 0.41
james_madison: 0.19
alexander_hamilton: 0.40

Predicting for Federalist Paper 30
Truth: alexander_hamilton
Prediction Summary:
thomas_jefferson: 0.39
james_madison: 0.17
alexander_hamilton: 0.43

Predicting for Federalist Paper 5
Truth: john_jay
Prediction Summary:
thomas_jefferson: 0.71
james_madison: 0.07
alexander_hamilton: 0.21

[epoch 2] Starting epoch 2
[batch 5633]: seen 3380400 words at 33802 wps, loss = 1.316
[epoch 2] Completed in 0:01:53
[epoch 2] Predicting for Federalist Paper 39
Truth: james_madison
Prediction Summary:
thomas_jefferson: 0.27
james_madison: 0.08
alexander_hamilton: 0.65

Predicting for Federalist Paper 30
Truth: alexander_hamilton
Prediction Summary:
thomas_jefferson: 0.31
james_madison: 0.03
alexander_hamilton: 0.66

Predicting for 

In [33]:
# Converting epoch predictions into a dataframe for analysis
if save_predictions:
    df = create_predictions_dataframe(epoch_predictions)

In [37]:
df

,18,19,20,49,50,51,52,53,54,55,56,57,58,62,63,epoch
1.0000,0.6167,0.6167,0.6500,0.4000,0.5875,0.3583,0.4750,0.4333,0.4833,0.4667,0.4750,0.4083,0.4083,0.4500,0.5400,1
3.0000,0.0833,0.1000,0.0875,0.1500,0.0750,0.2083,0.0750,0.1250,0.1500,0.1667,0.1375,0.2250,0.1500,0.1375,0.1500,1
4.0000,0.3000,0.2833,0.2625,0.4500,0.3375,0.4333,0.4500,0.4417,0.3667,0.3667,0.3875,0.3667,0.4417,0.4125,0.3100,1
1.0000,0.7000,0.5250,0.5625,0.2750,0.5250,0.2667,0.3750,0.3000,0.3500,0.4500,0.3750,0.3833,0.3667,0.3563,0.4100,2
3.0000,0.0667,0.0500,0.0375,0.1125,0.0375,0.1250,0.0583,0.0583,0.1417,0.0917,0.0875,0.1167,0.1167,0.1000,0.0750,2
4.0000,0.2333,0.4250,0.4000,0.6125,0.4375,0.6083,0.5667,0.6417,0.5083,0.4583,0.5375,0.5000,0.5167,0.5437,0.5150,2
1.0000,0.7083,0.5750,0.5875,0.3000,0.5625,0.3333,0.4417,0.3250,0.3833,0.4583,0.4125,0.4333,0.3750,0.3812,0.4350,3
3.0000,0.0667,0.0500,0.0625,0.1250,0.0875,0.1500,0.0833,0.0833,0.1667,0.1250,0.1000,0.1083,0.1417,0.1437,0.1000,3
4.0000,0.2250,0.3750,0.3500,0.5750,0.3500,0.5167,0.4750,0.5917,0.4500,0.4167,0.4875,0.4583,0.4833,0.4750,0.4650,3


In [44]:
print author_to_id['james_madison']
print author_to_id['george_washington']
print author_to_id

3
7
{'thomas_jefferson': 1, 'john_adams': 2, 'alexander_hamilton': 4, 'benjamin_franklin': 8, 'george_washington': 7, 'thomas_paine': 0, 'james_madison': 3, 'james_monroe': 5, 'john_jay': 6}
